Проверка на тестовом уравнении:

$y_1' = y_2 \\
y_2' = -y_1$

Аналитическое решение:

$y_1 = 2cos(t) \\
y_2 = -2sin(t)$

In [1]:
#начальные условия
y10 = 2.0
y20 = 0.0
T0 = 0.0
Tk = 20.0



## Метод Рунге-Кутты 4-го порядка:

In [2]:
%matplotlib inline
import scipy.optimize
import math
import numpy as np
import matplotlib.pyplot as plt

def f(y):
    y1 = y[0]
    y2 = y[1]
    f1 = y2
    f2 = -y1
    result = [f1, f2]
    return np.array(result)

def runge(f, y10, y20, T0, Tk, h):
    N = int(((Tk-T0)/h))
    y = [y10, y20] 
    Y_ = [y]
    y = np.array(y) 
    
    #Метод Рунге-Кутты 4-го порядка
    for i in range(N):
        k1 = f(y)
        k2 = f(y + h/2*k1)
        k3 = f(y + h/2*k2)
        k4 = f(y + h*k3)
        y = y + h/6*(k1 + 2*k2 + 2*k3 + k4)
        u = [y[0], y[1]]
        Y_.append(u)
    return Y_

## Неявный метод Эйлера

$\frac{y^{n+1}-y^{n}}{h} = f_{n+1}$

In [3]:
def jacobian(y, h):
    y1 = y[0]
    y2 = y[1]
    dfdy = np.zeros((2,2))
    dfdy[0][0] = 1
    dfdy[0][1] = -h
    dfdy[1][0] = h
    dfdy[1][1] = 1
    return dfdy

def F(y_curr, y_prev, h):
    return (y_curr - y_prev - h*f(y_curr))

In [4]:
def newton(y, h):
    eps = h**2
    max_iter = 7
    y_prev = y.copy()
    y_curr = y.copy()
    for i in range (max_iter):
        y_prev = y_curr.copy()
        delta = np.linalg.solve(jacobian(y_prev,h), F(y_prev, y, h))
        y_curr = y_curr - delta
        if (np.max(np.abs(delta)) < eps):
            break

    return y_curr

In [5]:
def eiler(f, y10, y20, T0, Tk, h):
    N = int((Tk - T0) / h)
    y = [y10, y20] 
    Y_ = [y]
    y = np.array(y)
    
    for i in range(N):
        y = newton(y,h)
        u = [y[0], y[1]]
        Y_.append(u)
    return Y_

## Неявная формула дифференцирования назад 3 порядка:


$\frac{11}{6}y^{n+1}-3y^{n}+\frac{3}{2}y^{n-1}-\frac{1}{3}y^{n-2}=hf_{n+1}$

Значения в точках $y^{1}, y^{2}$ найдем с помощью метода Рунге-Кутты 4 порядка

In [6]:
def newton2(y_pr3, y_pr2, y_pr1, h):
    eps = h**2
    max_iter = 5
    y_curr = y_pr1.copy()
    for i in range (max_iter):
        y_prev = y_curr.copy()
        delta = np.linalg.solve((jacobian2(y_prev,h)), F2(y_prev, y_pr3, y_pr2, y_pr1, h))
        y_curr = y_curr - delta
        if (np.max(np.abs(delta)) < eps):
            break
          
    return y_curr

def jacobian2(y, h):
    y1 = y[0]
    y2 = y[1]
    dfdy = np.zeros((2,2))
    dfdy[0][0] = 1
    dfdy[0][1] = -6/11 * h
    dfdy[1][0] = 6/11 * h
    dfdy[1][1] = 1
    return dfdy

def F2(y_prev, y_pr3, y_pr2, y_pr1, h):
    return (y_prev - 6/11*(3*y_pr1 - 1.5*y_pr2 + 1/3*y_pr3 + h*f(y_prev)))

In [7]:
def runge_for_bdf(y,h):
    k1 = f(y)
    k2 = f(y + h/2*k1)
    k3 = f(y + h/2*k2)
    k4 = f(y + h*k3)
    return(y + h/6*(k1 + 2*k2 + 2*k3 + k4))
    
def bdf(f, y10, y20, T0, Tk, h):
    N = int((Tk - T0) / h)
    y_pr3 = [y10, y20] #y_0
    y_pr2 = runge_for_bdf(y_pr3,h) #y_1
    y_pr1= runge_for_bdf(y_pr2,h) #y_2
    Y_ = [y_pr3]
    Y_.append([y_pr2[0], y_pr2[1]])
    Y_.append([y_pr1[0], y_pr1[1]])
    y_pr1 = np.array(y_pr1)
    y_pr2 = np.array(y_pr2)
    y_pr3 = np.array(y_pr3)

    for i in range(N):
        y = newton2(y_pr3, y_pr2, y_pr1, h)
        y_pr3 = y_pr2.copy()
        y_pr2 = y_pr1.copy()
        y_pr1 = y.copy()
        u = [y[0], y[1]]
        Y_.append(u)
    return Y_


## Оценка с помощью правила Рунге:

$a_1 = C h^p \\
a_2 = C (\frac{h}{2})^p \\
\Rightarrow p = \frac{log(a_2/a_1)}{log(2)}\\
\Rightarrow C = \frac{a_1}{h^p}$

In [8]:
def RUNGE_runge(h_r):
    Y_h = np.array(runge(f, y10, y20, T0, Tk, h_r))
    Y_05h = np.array(runge(f, y10, y20, T0, Tk, h_r/2))
    t_h = np.linspace(0, Tk, len(Y_h))
    t_05h = np.linspace(0, Tk, len(Y_05h))
    Y_h_tr = np.array(list(zip(2*np.cos(t_h), -2*np.sin(t_h))))
    Y_05h_tr = np.array(list(zip(2*np.cos(t_05h), -2*np.sin(t_05h))))
    a1 = np.linalg.norm(np.abs(Y_h - Y_h_tr), ord=2)
    a2 = np.linalg.norm(np.abs(Y_05h - Y_05h_tr), ord=2)
    p = np.abs(math.log2(a2/a1))
    C = a1/h_r**p
    return p, C

In [9]:
def RUNGE_eiler(h_r):
    Y_h = np.array(eiler(f, y10, y20, T0, Tk, h_r))
    Y_05h = np.array(eiler(f, y10, y20, T0, Tk, h_r/2))
    t_h = np.linspace(0, Tk, len(Y_h))
    t_05h = np.linspace(0, Tk, len(Y_05h))
    Y_h_tr = np.array(list(zip(2*np.cos(t_h), -2*np.sin(t_h))))
    Y_05h_tr = np.array(list(zip(2*np.cos(t_05h), -2*np.sin(t_05h))))
    a1 = np.max(np.abs(Y_h - Y_h_tr))
    a2 = np.max(np.abs(Y_05h - Y_05h_tr))
    p = np.abs(math.log2(a2/a1))
    C = a1/h_r**p
    return p, C

In [13]:
def RUNGE_bdf(h_r):
    Y_h = np.array(bdf_opt(f, y10, y20, T0, Tk, h_r))
    Y_05h = np.array(bdf_opt(f, y10, y20, T0, Tk, h_r/2))
    t_h = np.linspace(0, Tk, len(Y_h))
    t_05h = np.linspace(0, Tk, len(Y_05h))
    Y_h_tr = np.array(list(zip(2*np.cos(t_h), -2*np.sin(t_h))))
    Y_05h_tr = np.array(list(zip(2*np.cos(t_05h), -2*np.sin(t_05h))))
    a1 = np.linalg.norm(np.abs(Y_h - Y_h_tr), ord=2)
    a2 = np.linalg.norm(np.abs(Y_05h - Y_05h_tr), ord=2)
    p = np.abs(math.log2(a2/a1))
    C = a1/h_r**p
    return p, C

### Следующую ячейку лучше не запускать, потому что она очень-очень-очень долго считается

In [15]:
h_list = [10**(-3), 5*10**(-4), 2.5*10**(-4), 1.25*10**(-4), 6.25*10**(-5), 3.125*10**(-5)]
for h in h_list:
    print('h = ', h)
    print('RUNGE: p =', RUNGE_runge(h)[0], 'C =', RUNGE_runge(h)[1])
    print('EILER: p =', RUNGE_eiler(h)[0], 'C =', RUNGE_eiler(h)[1])
    print('BDF: p =', RUNGE_bdf(h)[0], 'C =', RUNGE_bdf(h)[1])
    print('---------------------------------------------------')

h =  0.001
RUNGE: p = 4.693535925516433 C = 3089.3644236232963
EILER: p = 0.9965696984135222 C = 18.34650562149176
BDF: p = 0.5000646140803456 C = 9.397270082112486
---------------------------------------------------
h =  0.0005
RUNGE: p = 2.122797525662968 C = 1.008752379311179e-05
EILER: p = 0.9982839105531659 C = 18.587116627513886
BDF: p = 0.5000323487559151 C = 9.39496572570303
---------------------------------------------------
h =  0.00025
RUNGE: p = 1.1178490237281802 C = 4.591464595150062e-08
EILER: p = 0.9991416949604025 C = 18.71982631090221
BDF: p = 0.5000161871436104 C = 9.393706459906214
---------------------------------------------------
h =  0.000125
RUNGE: p = 0.25668610731323194 C = 9.413845299043668e-11
EILER: p = 0.9995707829372779 C = 18.792154912257086
BDF: p = 0.5000080952259235 C = 9.393023340060308
---------------------------------------------------
h =  6.25e-05
RUNGE: p = 1.6070432701063706 C = 6.388089975299923e-05
EILER: p = 0.9997853753250692 C = 18.831232

## Сравнение с аналитическим решением

Метод Эйлера

In [8]:
for h in [0.0002, 0.0001, 0.00005]:
    N = int((Tk-T0)/h)
    Y_eiler = eiler(f, y10, y20, T0, Tk, h)
    y1_eiler = [Y_eiler[i][0] for i in range(len(Y_eiler))]
    y2_eiler = [Y_eiler[i][1] for i in range(len(Y_eiler))]
    r1 = y1_eiler[-1] - 2 * np.cos(N*h)
    r2 = y2_eiler[-1] + 2 * np.sin(N*h)
    print('T =', N*h)
    print('h =', h, 'r1 =', abs(r1))
    print('h =', h, 'r2 =', abs(r2))
    print('-------------------------------')

T = 20.0
h = 0.0002 r1 = 0.001630211042860652
h = 0.0002 r2 = 0.003648348791329026
-------------------------------
T = 20.0
h = 0.0001 r1 = 0.0008156345691151534
h = 0.0001 r2 = 0.0018250322078527947
-------------------------------
T = 20.0
h = 5e-05 r1 = 0.0004079496414900463
h = 5e-05 r2 = 0.0009127306472387353
-------------------------------


ФДН 3 порядка

In [9]:
for h in [0.0002, 0.0001, 0.00005]:
    N = int((Tk-T0)/h)
    Y_bdf = bdf(f, y10, y20, T0, Tk, h)
    y1_bdf = [Y_bdf[i][0] for i in range(len(Y_bdf))]
    y2_bdf = [Y_bdf[i][1] for i in range(len(Y_bdf))]
    r1 = y1_bdf[-1] - 2 * np.cos(N*h)
    r2 = y2_bdf[-1] + 2 * np.sin(N*h)
    print('T =', N*h)
    print('h =', h, 'r1 =', abs(r1))
    print('h =', h, 'r2 =', abs(r2))
    print('-------------------------------')

T = 20.0
h = 0.0002 r1 = 0.000730421454155139
h = 0.0002 r2 = 0.00032631961436502266
-------------------------------
T = 20.0
h = 0.0001 r1 = 0.00036519444209548446
h = 0.0001 r2 = 0.00016319625878313815
-------------------------------
T = 20.0
h = 5e-05 r1 = 0.0001825931805538028
h = 5e-05 r2 = 8.1607171482867e-05
-------------------------------


Метод Рунге-Кутты 4 порядка

In [10]:
for h in [0.001, 0.0005, 0.00025]:
    N = int((Tk-T0)/h)
    Y_runge = runge(f, y10, y20, T0, Tk, h)
    y1_runge = [Y_runge[i][0] for i in range(len(Y_runge))]
    y2_runge = [Y_runge[i][1] for i in range(len(Y_runge))]
    r1 = y1_runge[-1] - 2 * np.cos(N*h)
    r2 = y2_runge[-1] + 2 * np.sin(N*h)
    print('T =', N*h)
    print('h =', h, 'r1 =', abs(r1))
    print('h =', h, 'r2 =', abs(r2))
    print('-------------------------------')

T = 20.0
h = 0.001 r1 = 3.199662756969701e-13
h = 0.001 r2 = 1.2367884494324244e-13
-------------------------------
T = 20.0
h = 0.0005 r1 = 2.220446049250313e-16
h = 0.0005 r2 = 3.552713678800501e-15
-------------------------------
T = 20.0
h = 0.00025 r1 = 1.509903313490213e-14
h = 0.00025 r2 = 2.2870594307278225e-14
-------------------------------
